In [ ]:
from zipfile import ZipFile
from pathlib import Path
from matplotlib import font_manager
from urllib.request import urlretrieve

if not Path('./Fira-4.202').is_dir():
    urlretrieve("https://github.com/mozilla/Fira/archive/refs/tags/4.202.zip", "./Fira-4.202.zip")
    with ZipFile('./Fira-4.202.zip', 'r') as zip:
        zip.extractall('.')
    with open('./Fira-4.202/.gitignore', 'w') as ignore:
        ignore.write('*')

font_dirs = ["./Fira-4.202/ttf/"]  # The path to the custom font file.
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob

sns.set_theme(style="ticks", font="Fira Sans")

frames = [
    pd.read_csv(
        data,
        names=[
            "protocol",
            "app",
            "drop rate",
            "number of clients",
            "number of faulty",
            "client host index",
            "throughput",
            "average latency"],
        keep_default_na=False)
    for data in glob("../data/saved-*.csv")]
data = pd.concat(frames, ignore_index=True)
saved_data = data
data

In [ ]:
data = saved_data
p = (data['app'] == 'null') & (data['drop rate'] == 0.) & (data['number of faulty'] == 1)
g = sns.relplot(
    data=pd.DataFrame(saved_data)[p], 
    kind="line",
    sort=False,
    x="throughput", y="average latency", hue="protocol",
    hue_order=['unreplicated', 'neo-hm', 'neo-pk', 'neo-bn', 'pbft', 'zyzzyva', 'zyzzyva-f', 'hotstuff', 'minbft'],
    height=2.56, aspect=2.)
g.set(ylim=(0, 6000))
# g.set(ylim=(0, 2000))

In [ ]:
g = sns.catplot(
    data=data[data['app'] == 'ycsb'], 
    kind="bar",
    x="protocol", y="throughput",
    order=['unreplicated', 'neo-hm', 'neo-pk', 'neo-bn', 'pbft', 'zyzzyva', 'zyzzyva-f', 'hotstuff', 'minbft'],
    height=2.56, aspect=2.)
g.set_xticklabels(rotation=30)

In [ ]:
g = sns.relplot(
    data=data[(data['app'] == 'null') & (data['drop rate'] != 0.)], 
    kind="line",
    x="drop rate", y="throughput", hue="protocol",
    height=2.56, aspect=2.)
g.set(xscale='log')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes 
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

In [ ]:
lstyles = ['-', '--', '-.', '-']
xfiles = [ '25% load', '50% load', '99% load']
files = [ 'results_fpga25.log', 'results_fpga50.log', 'results_fpga99.log' ]
files = sorted(files)

fig, ax = plt.subplots()
res = []

for i, ff in enumerate(files):
    with open(f'../data/{ff}') as f:
        data = f.readlines()
    data = [ d.strip() for d in data if d.strip() ]
    data = data[:1000000]
    data = [ d.replace(':', '') for d in data ]
    data = [ float(int(d, 16) / 1000) for d in data ]
    data = sorted(data)
    
    res.append(data)
    
    print("25.0th", int(np.percentile(data, 25)), "us")
    print("50.0th", int(np.percentile(data, 50)), "us")
    print("99.0th ", int(np.percentile(data, 99)), "us")
    print("99.9th", int(np.percentile(data, 99.9)), "us")
    print("99.99th", int(np.percentile(data, 99.99)), "us")
    print("99.999th", int(np.percentile(data, 99.999)), "us")
    
    # sns.kdeplot(data = data, cumulative = True, label = ff, )
    y = 1. * np.arange(len(data)) / (len(data) - 1)
    plt.plot(data, y, lstyles[i%4], label=xfiles[i])
    # plt.plot(data, y)
plt.tick_params(direction='in')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Latency (us)", fontsize=14)
plt.ylabel("CDF", fontsize=14)
plt.legend(fontsize=14)
plt.tight_layout()

axins = zoomed_inset_axes(ax, 10, loc='center')
axins.set_xlim([2.9, 2.94]) # Limit the region for zoom
axins.set_ylim([0.98, 1.01])
axins.set_yticks([0.98, 0.99, 1.0])

axins.tick_params("x", labelsize=14)
axins.tick_params("y", labelsize=14)
# plt.xticks(visible=False)  # Not present ticks
# plt.yticks(visible=False)
## draw a bbox of the region of the inset axes in the parent axes and
## connecting lines between the bbox and the inset axes area
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")
for i, data in enumerate(res):
    y = 1. * np.arange(len(data)) / (len(data) - 1)
    axins.plot(data, y, lstyles[i%4], label=xfiles[i])

In [ ]:
lstyles = ['-', '--', '-.', '-']

xfiles = [ '25% load', '50% load', '99% load']
files = [ 'results_hmac25.log', 'results_hmac50.log', 'results_hmac99.log' ]
files = sorted(files)

fig, ax = plt.subplots()
res = []

for i, ff in enumerate(files):
    with open(f'../data/{ff}') as f:
        data = f.readlines()
    data = [ d.strip() for d in data ]
    data = data[:1000000]
    data = [ d.replace(':', '') for d in data ]
    data = [ float(int(d, 16) / 1000) for d in data ]
    data = sorted(data)
    
    res.append(data)
    
    print("25.0th", int(np.percentile(data, 25)), "us")
    print("50.0th", int(np.percentile(data, 50)), "us")
    print("99.0th ", int(np.percentile(data, 99)), "us")
    print("99.9th", int(np.percentile(data, 99.9)), "us")
    print("99.99th", int(np.percentile(data, 99.99)), "us")
    print("99.999th", int(np.percentile(data, 99.999)), "us")
    
    # sns.kdeplot(data = data, cumulative = True, label = ff, )
    y = 1. * np.arange(len(data)) / (len(data) - 1)
    plt.plot(data, y, lstyles[i%4], label=xfiles[i])
    # plt.plot(data, y)  
plt.tick_params(direction='in')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Latency (us)", fontsize=14)
plt.ylabel("CDF", fontsize=14)
plt.legend(fontsize=14)
plt.tight_layout()

axins = zoomed_inset_axes(ax, 30, loc='center')
axins.set_xlim([9.0, 9.6]) # Limit the region for zoom
axins.set_ylim([0.99, 1.002])
axins.set_yticks([0.99, 0.995, 1.0])
axins.set_yticks([0.99, 1.0])

axins.tick_params("x", labelsize=14)
axins.tick_params("y", labelsize=14)
# plt.xticks(visible=False)  # Not present ticks
# plt.yticks(visible=False)
## draw a bbox of the region of the inset axes in the parent axes and
## connecting lines between the bbox and the inset axes area
mark_inset(ax, axins, loc1=2, loc2=4, fc="none", ec="0.5")
for i, data in enumerate(res):
    y = 1. * np.arange(len(data)) / (len(data) - 1)
    axins.plot(data, y, lstyles[i%4], label=xfiles[i])

In [ ]:
hmac_number = 76.25
fpga_number = 1.11

num = 64
num_replica = [ i for i in range(4, num+4, 4)]
hmac_tput = []
peak_tput = hmac_number
for i, replica in enumerate(num_replica):
    hmac_tput.append(peak_tput/(i+1))
fpga_tput = [fpga_number for i in num_replica]

print(num_replica)
print(hmac_tput)
print(fpga_tput)

plt.plot(num_replica, fpga_tput, label='AOM-PK', marker="^")
plt.plot(num_replica, hmac_tput, label='AOM-HM', marker=".")
# plt.plot(num_replica, hmac_tput, label='Neo-HM')
# plt.plot(num_replica, fpga_tput, label='Neo-PK')

plt.tick_params(direction='in')
# plt.yscale('log')
# plt.yticks([100, 1000, 10000])
plt.annotate(str(fpga_number) + "Mpps", (2,3),fontsize=14)
plt.annotate(str(hmac_number) + "Mpps", (5,75),fontsize=14)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

plt.xlabel('Number of Receivers',fontsize=14)
plt.ylabel('Throughput (Mpps)', fontsize=14)

plt.legend(fontsize=14)

plt.tight_layout()